In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional, Reshape, Conv2D, MaxPool2D, Concatenate, Flatten, MaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
size_vect = 768

def label_to_one_hot(label):
  one_hot = [0, 0, 0]
  one_hot[label] = 1
  return one_hot

def prepare_data_bert(data):
  
  data["sentence_embedding"] = data["sentence_embedding"].apply(lambda x : np.asarray([float(value) for value in x[1:-1].split(",")]).astype(np.float32))

  x_train, x_test, _, y_test = train_test_split(data['sentence_embedding'], data['label'], test_size=0.3, random_state=42)

  x_train_to_return = np.zeros(shape=(len(x_train), 768))
  for i in range(len(x_train)):
      x_train_to_return[i] = x_train["sentence_embedding"].values[i]

  x_test_to_return = np.zeros(shape=(len(x_test), 768))
  for i in range(len(x_test)):
      x_test_to_return[i] = x_test["sentence_embedding"].values[i]

  data["yhot"] = data["label"].apply(label_to_one_hot)
  y_train = np.zeros(shape=(len(data), 3))
  for i in range(len(data)):
      y_train[i] = data["yhot"].values[i]
  y_train = y_train.astype(int)

  return x_train_to_return, x_test_to_return, y_train, y_test


In [ ]:
def cnn_1(size_vector=768, metric='accuracy'):
  filter_sizes = [5,50,3]
  num_filters = 10
  drop = 0.9
  batch_size = 15
  epochs = 2

  inputs = Input(shape=(size_vector,1), dtype='float32')
  conv_0 = Conv1D(num_filters, kernel_size=filter_sizes[0], padding='valid', kernel_initializer='normal', activation='relu')(inputs)
  conv_1 = Conv1D(num_filters, kernel_size=filter_sizes[1], padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_2 = Conv1D(num_filters, kernel_size=filter_sizes[2], padding='valid', kernel_initializer='normal', activation='relu')(reshape)

  maxpool_0 = MaxPool1D(pool_size=2, padding='same')(conv_0)
  maxpool_1 = MaxPool1D(pool_size=(300 - filter_sizes[1] + 1), padding='valid')(conv_1)
  maxpool_2 = MaxPool1D(pool_size=(300 - filter_sizes[2] + 1), padding='valid')(conv_2)

  concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
  flatten = Flatten()(concatenated_tensor)
  dropout = Dropout(drop)(flatten)
  # dense = Dense(units=294912, activation='softmax')(dropout)
  output = Dense(units=3, activation='softmax')(dropout)
  model = Model(inputs=inputs, outputs=output)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metric])
  return model

def cnn_2(size_vector=768, metric='accuracy'):
  filter_sizes = [5,50,3]
  num_filters = 10
  drop = 0.9
  batch_size = 15
  epochs = 2

  inputs = Input(shape=(size_vector,1), dtype='float32')
  conv_0 = Conv2D(num_filters, kernel_size=filter_sizes[0], padding='valid', kernel_initializer='normal', activation='relu')(inputs)
  conv_1 = Conv2D(num_filters, kernel_size=filter_sizes[1], padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_2 = Conv2D(num_filters, kernel_size=filter_sizes[2], padding='valid', kernel_initializer='normal', activation='relu')(reshape)

  maxpool_0 = MaxPool2D(pool_size=2, padding='same')(conv_0)
  maxpool_1 = MaxPool2D(pool_size=(300 - filter_sizes[1] + 1), padding='valid')(conv_1)
  maxpool_2 = MaxPool2D(pool_size=(300 - filter_sizes[2] + 1), padding='valid')(conv_2)

  concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
  flatten = Flatten()(concatenated_tensor)
  dropout = Dropout(drop)(flatten)
  output = Dense(units=3, activation='softmax')(dropout)
  model = Model(inputs=inputs, outputs=output)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metric])
  return model

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K

class attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

def lstm(size_vector=768):
  model = Sequential()
  model.add(Input(shape=(size_vector,1), dtype='float32'))
  model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy']) 
  return model

def lstm_attention(size_vector=768, metric='accuracy'):
  model = Sequential()
  model.add(Input(shape=(size_vector,1), dtype='float32'))
  model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
  model.add(attention()) 
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy']) 
  return model

In [ ]:
data_bert = pd.read_csv("/content/drive/MyDrive/dataset_watson/train_embedded_bert.csv")
x_train, x_test, y_train, y_test = prepare_data_bert(data)

In [ ]:
def train_plot(x_train, y_train, model, model_name, metric="accuracy", batch_size = 64, epochs = 200):
    history = model.fit(x_train, y_train, validation_split=0.15, batch_size=batch_size, epochs=epochs, verbose=True)
    # list all data in history
    print(history.history.keys())

    # summarize history for accuracy
    plt.plot(history.history[metric.lower()], c="blue", label="train")
    plt.plot(history.history['val_'+metric.lower()], c="orange", label="test")
    plt.title('model '+metric)
    plt.ylabel(metric)
    plt.xlabel('epoch')
    plt.legend(loc='upper left')
    filename = model_name + '_metric_Bert.jpg'
    fig.savefig(filename, bbox_inches='tight', dpi=150)
    plt.show()
    

    # summarize history for loss
    plt.plot(history.history['loss'], c="blue", label="train")
    plt.plot(history.history['val_loss'], c="orange", label="test")
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    filename = model_name + '_loss_Bert.jpg'
    fig.savefig(filename, bbox_inches='tight', dpi=150)
    plt.show()

    return model, history

In [ ]:
CNN_1D_acc = cnn_1('accuracy')
model_cnn1_acc,history_cnn1_acc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=CNN_1D_acc,
                           model_name='CNN_1D_acc'
                           metric='accuracy',
                           batch_size=64,
                           epochs=100)

CNN_1D_auc = cnn_1('AUC')
model_cnn1_auc,history_cnn1_auc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=CNN_1D_auc,
                           model_name='CNN_1D_auc'
                           metric='AUC',
                           batch_size=64,
                           epochs=100)

In [ ]:
CNN_2D_acc = cnn_2('accuracy')
model_cnn2_acc,history_cnn2_acc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=CNN_2D_acc,
                           model_name='CNN_2D_acc'
                           metric='accuracy',
                           batch_size=64,
                           epochs=100)

CNN_2D_auc = cnn_2('AUC')
model_cnn2_auc,history_cnn2_auc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=CNN_2D_auc,
                           model_name='CNN_2D_auc'
                           metric='AUC',
                           batch_size=64,
                           epochs=100)

In [ ]:
lstm_acc = lstm('accuracy')
model_cnn2_acc,history_cnn2_acc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=lstm_acc,
                           model_name='lstm_acc'
                           metric='accuracy',
                           batch_size=64,
                           epochs=100)

lstm_auc = lstm('AUC')
model_cnn2_auc,history_cnn2_auc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=lstm_auc,
                           model_name='lstm_auc'
                           metric='AUC',
                           batch_size=64,
                           epochs=100)

In [ ]:
lstm_att_acc = lstm('accuracy')
model_cnn2_acc,history_cnn2_acc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=lstm_att_acc,
                           model_name='lstm_att_acc'
                           metric='accuracy',
                           batch_size=64,
                           epochs=100)

lstm_att_auc = lstm('AUC')
model_cnn2_auc,history_cnn2_auc = train_plot(x_train=x_train,
                           y_train=y_train,
                           model=lstm_att_auc,
                           model_name='lstm_att_auc'
                           metric='AUC',
                           batch_size=64,
                           epochs=100)